In [2]:
import numpy as np
import pandas as pd
import requests
import json

In [3]:
card_list = pd.read_csv('yugioh.csv')

In [4]:
card_list

,card_name
0,Tornado Dragon
1,Exodia the Forbidden One
2,Dark Magician
3,Kuriboh
4,Obelisk The Tormentor
5,Pot of Greed
6,Yami
7,Mystical Elf
8,Monster Reborn
9,Slifer the Sky Dragon


In [6]:
card_list = card_list.card_name.to_list()

In [8]:
card_list

['Tornado Dragon',
 'Exodia the Forbidden One',
 'Dark Magician',
 'Kuriboh',
 'Obelisk The Tormentor',
 'Pot of Greed',
 'Yami',
 'Mystical Elf',
 'Monster Reborn',
 'Slifer the Sky Dragon']

In [9]:
params = {'name': ['Tornado Dragon']}
endpoint_get = 'https://db.ygoprodeck.com/api/v7/cardinfo.php'
r = requests.get(endpoint_get, params = params)

In [11]:
print(r.json())

{'data': [{'id': 6983839, 'name': 'Tornado Dragon', 'type': 'XYZ Monster', 'desc': '2 Level 4 monsters\nOnce per turn (Quick Effect): You can detach 1 material from this card, then target 1 Spell/Trap on the field; destroy it.', 'atk': 2100, 'def': 2000, 'level': 4, 'race': 'Wyrm', 'attribute': 'WIND', 'card_sets': [{'set_name': 'Battles of Legend: Relentless Revenge', 'set_code': 'BLRR-EN084', 'set_rarity': 'Secret Rare', 'set_rarity_code': '(ScR)', 'set_price': '4.65'}, {'set_name': 'Duel Devastator', 'set_code': 'DUDE-EN019', 'set_rarity': 'Ultra Rare', 'set_rarity_code': '(UR)', 'set_price': '1.58'}, {'set_name': 'Legendary Duelists: Synchro Storm', 'set_code': 'LED8-EN055', 'set_rarity': 'Common', 'set_rarity_code': '(C)', 'set_price': '0.96'}, {'set_name': 'Maximum Crisis', 'set_code': 'MACR-EN081', 'set_rarity': 'Secret Rare', 'set_rarity_code': '(ScR)', 'set_price': '4.49'}], 'card_images': [{'id': 6983839, 'image_url': 'https://images.ygoprodeck.com/images/cards/6983839.jpg', 

In [ ]:
# Mencoba commit and push to main branch